In [1]:
import urllib.request 
import re
import numpy as np

In [2]:
alice_http_response = urllib.request.urlopen('https://www.cs.cmu.edu/~rgs/alice-I.html')
alice_line_by_line = [line.decode('utf-8') for line in alice_http_response]

In [3]:
alice_line_by_line[0:20]

['<HTML>\n',
 '<HEAD>\n',
 "<Title>Alice's Adventures in Wonderland -- Chapter I</Title>\n",
 '</HEAD>\n',
 '<BODY>\n',
 '<H2>CHAPTER I</H2>\n',
 '                      <H2>Down the Rabbit-Hole</H2>\n',
 '\n',
 '  Alice was beginning to get very tired of sitting by her sister\n',
 'on the bank, and of having nothing to do:  once or twice she had\n',
 'peeped into the book her sister was reading, but it had no\n',
 "pictures or conversations in it, `and what is the use of a book,'\n",
 "thought Alice `without pictures or conversation?'\n",
 '<P>\n',
 '  So she was considering in her own mind (as well as she could,\n',
 'for the hot day made her feel very sleepy and stupid), whether\n',
 'the pleasure of making a daisy-chain would be worth the trouble\n',
 'of getting up and picking the daisies, when suddenly a White\n',
 'Rabbit with pink eyes ran close by her.\n',
 '<P>\n']

In [4]:
%%time
alice_clean = [re.sub('<.+?>', '', line) for line in alice_line_by_line]
# note that unlike + which is a gready qualifier 
# +? is a lazy qualifier that stops after the first match
alice_clean = [re.sub('\n', '', line) for line in alice_clean]
alice_clean = [re.sub('^\s+', '', line) for line in alice_clean]
alice_clean = [re.sub('\*.+\*', '', line) for line in alice_clean]
alice_clean = [line for line in alice_clean if len(line)>0]

alice_single_text = ' '.join(alice_clean)

CPU times: total: 0 ns
Wall time: 997 µs


In [5]:
alice_single_text[0:500]

"Alice's Adventures in Wonderland -- Chapter I CHAPTER I Down the Rabbit-Hole Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do:  once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?' So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whe"

In [6]:
len(alice_single_text)

11456

In [7]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


# Solutions


### 1. What is the most lengthy sequence that repeats twice or more?

In [36]:
%%time
# there are quite a few nice ways to solve it. I picked a solution that is based on modifying Edit Distance / Dynamic programming
mat = np.zeros([len(alice_single_text), len(alice_single_text)])
for i in range(len(alice_single_text)):
    for j in range(0, len(alice_single_text)):
        if i==j:
            continue
        if alice_single_text[i] == alice_single_text[j]:
            if i==0 or j==0:
                mat[i, j] = 1 
            else:
                mat[i, j] = mat[i-1, j-1] + 1
        else:
            mat[i, j] = 0 

print(np.max(mat))

27.0
CPU times: total: 52.9 s
Wall time: 52.9 s


In [37]:
# and here is the text (with extra char at the begining and the end)
pair_len = int(np.max(mat))
y_axis = int(np.argmax(mat) / len(alice_single_text))
x_axis = np.argmax(mat) % len(alice_single_text)
print(pair_len, y_axis, x_axis)
print(alice_single_text[y_axis-pair_len:y_axis+2])
print(alice_single_text[x_axis-pair_len:x_axis+2])

27 7347 9592
o she went back to the table,
n she went back to the table 


In [38]:
for i in range(y_axis-pair_len-5,y_axis+5):
    for j in range(x_axis-pair_len-5,x_axis+5):
        print(f'{mat[i,j].astype(int):2}', end = ' ')
    print()

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0 
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2 
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
 0  1  0  0  0  0  1  0  0  0  1  0  0  0  0  1  0  0  0  0  1  0  0  1  0  0  0  1  0  0  0  0  0  1  0  0  0 
 0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0 
 0  1  0  0  0  0  1  0  0  0  1  0  0  0  0  1  0  0  0  0  1  0  0  2  0  0  0  1  0  0  0  0  0  1  0  0  0 
 0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
 0  0  0  1  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0

### 2. Compare the run time of Aho-Corasick dictionary matching algorithm with a naive approach for many words


In [32]:
import ahocorasick

dict_matching = ahocorasick.Automaton()

set_of_words_to_find = set(re.sub('\W+', '', x) for x in alice_single_text.split())

print(len(set_of_words_to_find))

for idx, key in enumerate(set_of_words_to_find):
    dict_matching.add_word(key, (idx, key))

dict_matching.make_automaton()

649


In [33]:
%%time
count = 0
for end_index, (insert_order, original_value) in dict_matching.iter(alice_single_text):
    #if count < 10:
    #    start_index = end_index - len(original_value) + 1
    #    print((start_index, end_index, (insert_order, original_value)))
    count += 1
    
print(count)

4109
CPU times: total: 0 ns
Wall time: 1.99 ms


In [34]:
%%time
count = 0
for item in set_of_words_to_find:
    tmp = [m.start() for m in re.finditer(item, alice_single_text)]
    count += len(tmp)
print(count)

15566
CPU times: total: 31.2 ms
Wall time: 33 ms


### 3. Print the different GPEs (Geo-Political Entities) in Alice's story

In [35]:
doc = nlp(alice_single_text)
print([X.text for X in doc.ents if X.label_=='GPE'])

['Wonderland', 'New Zealand', 'Australia', 'Dinah', 'turkey']
